In [14]:
!pip install pandas
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

# Imports

In [496]:
#HTMl
from bs4 import BeautifulSoup
import html5lib
import requests

#Calcs
import pandas as pd
import locale #nomalize numbers

#Graphs
from bokeh.charts import Bar, Donut, output_notebook, show
from bokeh.layouts import gridplot, row
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs

# Pega o site

In [350]:
url = "http://www.transparencia.gov.br/PortalComprasDiretasOEUnidadeGestora.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243"

response = requests.get(url)

content = response.content

soup = BeautifulSoup(content, "html.parser")

In [351]:
a_tags = soup.find_all("a")
cont = 0;
for link in a_tags:
    if "PortalComprasDiretasOEElementoDespesa" in link.get("href"):
        print(link.get("href"))
    cont += 1

print(cont)

PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153109
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153107
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153253
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153113
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153977
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153103
23


# Funções de tratamentos dos dados

In [447]:
####IVAN

#Funções para tratamento das strings
#Remove o "." dos numeros(strings) na tabela e corrige os espaços
#causados por diferença de codificação ( \xa0 -> ' ')
def fix_str(val):
    for char in val:
        if "." in char:
            val = val.replace(char,'')
        elif "\xa0" in char:
            val = val.replace(char,' ')
    return val
#Converte a string da tabela para um float e adiciona as duas casas
#decimais após a virgula.
def fix_decimal_n_float(val):
    return float(val)/100

#lê URL, cria dataframe do pandas. Essa função cria um array de dataframes. Logo, se um site
#possui mais de uma tabela, elas serão agrupadas em forma de array.
def retorna_tabelas(val):
    return pd.read_html(val)

#Percorre a tabela e converte o string para float
#Passe como parâmetro dessa função a tabela que você quer normalizar e o nome da coluna
#que será convertida para float.
def corrige_tabela(table, column_name):
    for atual in range(len(table)):
        table.iloc[atual][column_name] = fix_decimal_n_float(fix_str(
                                        table.iloc[atual][column_name]))
        
#Cria a tabela a partir do DataFrame lido do HTML
#Passe como entrada o dataframe e a função retornará a tabela pronta, com as colunas
#nomeadas.
def cria_tabela(df, inicio=0, fim=0):
    col_vet = []
    #se o range de copia não é definido, o função percorre até
    #o final da tabela
    if fim == 0 :
        fim = len(df)
        
    #Nomeia as tabelas com numero de colunas variado
    for atual in range(0, df.columns.values.size):
        col_vet.append(fix_str(df[atual][0]))  
            
    
    temp = df[inicio:fim].copy()
    temp.columns = col_vet
    return temp

####IVAN

In [359]:
####IVAN
dfs = retorna_tabelas(url)

#Tabela com todo o dinheiro distribuído para a educação e quanto é
#destinado a UFRN
# não dá para usar o cria tabela, pois os valores 'numericos', já são float MUITO ESTRANHO
Gastos_totais = dfs[0][1:].copy()
Gastos_totais.columns = [dfs[0][0][0], "Moéda", dfs[0][1][0]]

#Tabela com o dinheiro da UFRN e como ele é distribuído entre as unidades
Gastos_por_unidade = cria_tabela(dfs[1], 1)
Gastos_por_unidade.drop("Código", axis=1, inplace=True)

#Percorre a tabela e converte o string para float
#O e+ é uma reação padrão da linguagem, para que os valores caibam na tabela
#Ao acessar cada valor individualmente, é possivel pegar o valor correto e completo
corrige_tabela(Gastos_por_unidade, "Total no Ano (R$)")

####IVAN

In [361]:
Gastos_por_unidade

,Unidade Gestora,Total no Ano (R$)
1,CENTRO DE BIOCIENCIAS DA UFRN,2.18907e+06
2,CENTRO DE CIENCIAS DA SAUDE DA UFRN,3.73863e+06
3,CENTRO DE CIENCIAS HUM.E LETRAS ARTES DA UFRN,2.56673e+06
4,CENTRO DE CIENCIAS SOCIAIS APLICADAS DA UFRN,1.26147e+06
5,CENTRO DE ENSINO SUPERIOR DO SERIDO/UFRN,731373
6,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,1.40942e+09


# Plotagem

In [376]:
####IVAN


#Verificar se esse gráfico é mesmo necessário e solucionar o problema
# da diferença de valores
graph = Donut(Gastos_por_unidade[:5], label="Unidade Gestora", values="Total no Ano (R$)", title="Distribuição do dinheiro" )
graph.add_tools(HoverTool(tooltips=[("Unidade:", "@{Unidade Gestora}"),
                                    ("Total:", "@{Total no Ano (R$)}")
                                   ]
                         ))

#Diferença de valores muito grande. :/
graph2 = Bar(Gastos_por_unidade, values="Total no Ano (R$)", label="Unidade Gestora", color="Unidade Gestora", title="Distribuição do dinheiro")

layout = row([graph, graph2], sizing_mode="scale_width")
output_notebook()
show(layout)

####IVAN

Loading BokehJS ...

In [453]:
####IVAN

##Percorre os links e cria tabelas para cada um deles

### Aqui que entra o SCRAP
urls = []
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153109")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153107")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153253")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153113")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153977")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153103")

##Lista de tabelas
Departamentos = []
#Criar um dicionário da seguinte form dict = { "Nome_Do_departamento" : Tabela}, assim fica mais fácil de 
#identificar cada gráfico que será plotado
for atual in range(len(urls)):
    df = retorna_tabelas(urls[atual])
    Departamentos.append(cria_tabela(df[1], 1))

####IVAN

In [456]:
####IVAN
for atual in range(len(Departamentos)):
    corrige_tabela(Departamentos[atual], "Total no Ano (R$)" )
    
####IVAN

In [497]:
Graficos = []

for atual in range(len(Departamentos)):
    graph = Bar(Departamentos[atual], values="Total no Ano (R$)", label="Elemento de Despesa", color="Elemento de Despesa",
                title=str(atual)+" Inisa o nome da UNIDADE", legend=None)
    graph.add_tools(HoverTool(tooltips=[("Gasto", "$y{f.ff}")]))
    Graficos.append(graph)

grfs = [Graficos[0:2], Graficos[2:4], Graficos[4:6]]

layout = gridplot(grfs, sizing_mode='scale_width')
output_notebook()

show(layout)

Loading BokehJS ...